# Lab 5: PageRank, TrustRank, HITS

### Before you start

- Notebook tasks can be completed individually or in a group of two
- Please save your notebooks with filled cell outputs; this will speed up the checking process
- Send notebook with solutions via e-mail:
  - To: michal.wojcik@doctorate.put.poznan.pl
  - Subject format example: [IR] Lab 5 - Jan Kowalski 123456, Anna Nowak 789012
  - Attachment: notebook file
- **Deadline:** 14 days after the class
- Some of the tasks require implementation - complete the code
- Some of the tasks require answering questions - answer them in Markdowns (just below the questions)
- The number of points for each task is given next to the command

In [ ]:
import networkx as nx
import pickle
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

### Task 1 - graphs definition [4p]

Prepare some graphs with the *networkx* library as described. Show:
- list of nodes
- adjacency matrix
- draw a graph

**a) *Random* graph: [1p]**

Create directed graph ([nx.DiGraph](https://networkx.org/documentation/stable/reference/classes/digraph.html)) with:
  - 10 nodes
  - 25 edges (generated randomly or manually; self-loops are possible but not required)
  - *connected* (according to the default concept of conectivity - there is at least one path connecting each pair of nodes; let's additionally assume that not all of the paths need to be *directed*)

**NOTE:** set the random seed so that the graph is reproducible or declare the edges manually

In [ ]:
# G_random = ...

print(G_random.nodes)
print(nx.adjacency_matrix(G_random).todense())
nx.draw(G_random, with_labels=True)

**b) *Spider-trap* graph: [1p]**

Create directed graph with:
- at least 5 nodes
- at least 2 nodes in *spider-trap(s)* (two separate *traps* or one *trap* with two nodes)

**NOTE:** declare the edges manually

In [ ]:
# G_spider_trap = ...

print(G_spider_trap.nodes)
print(nx.adjacency_matrix(G_spider_trap).todense())
nx.draw(G_spider_trap, with_labels=True)

**c) *Dead-end* graph: [1p]**

Create directed graph with:
- at least 5 nodes
- 1 or 2 nodes as a *dead-end(s)*

**NOTE:** declare the edges manually

In [ ]:
# G_dead_end = ...

print(G_dead_end.nodes)
print(nx.adjacency_matrix(G_dead_end).todense())
nx.draw(G_dead_end, with_labels=True)

**d) *Complete* graph: [1p]**

Create directed graph with:
- at least 5 nodes (**Hint:** 8 is drawn nicely)
- connection for every pair of distinct nodes (in both directions)

**NOTE:** declare the edges manually or use a dedicated method

In [ ]:
# G_complete = ...

print(G_complete.nodes)
print(nx.adjacency_matrix(G_complete).todense())
nx.draw(G_complete, with_labels=True)

### Task 2 - PageRank algorithm implementation [10p]

Implement two approaches to the PageRank algorithm.

Below is a reference function from the *networkx* package to verify the operation of the implemented functions

In [ ]:
def networkx_pagerank(graph: nx.DiGraph, alpha: float = 0.85) -> list:
    # graph - NetworkX Graph
    # alpha - damping parameter (opposite to damping factor from the lecture presentation)
    
    # PageRank by NetworkX library
    pagerank = nx.pagerank(graph, alpha=alpha)

    # Sort dictionary by PageRank value
    pagerank_sorted = sorted(pagerank.items(), key=lambda v:(v[1],v[0]), reverse=True)
    return pagerank_sorted

networkx_pagerank(G_random)

**a) *Random walk* approach [5p]**

Create a function that will estimate the PageRank value for each node in the graph using the "Random walk" strategy

- Randomly select start node
- In each iteration, with probability equal to:
  - $\alpha$ - go to one of the next nodes in the graph - randomly select one of the successors with equal probability
  - $1 - \alpha$ - go to a random node in the graph - simulate a situation when a user starts browsing from a new, randomly selected page
- If the algorithm reaches a *dead end*, go to a random node in the graph
- Count the occurrences of each node on the random walk (and normalize at the end to 1)
- Return the result in the same way as the *networkx_pagerank* function - a list of tuples sorted in descending order of PageRank values: (node_name, node_pagerank_value)
- Verify the method for different *alpha* values and graphs (random graph, dead end, spider trap, complete graph)

**NOTE:** for the sake of randomness, this result will be an approximation, but won't be exactly the same as in the *networkx_pagerank* function

In [ ]:
def randomwalk_pagerank(graph: nx.DiGraph, alpha: float = 0.85) -> list:
#     ...

randomwalk_pagerank(G_random)

**b) *Stochastic matrix* approach [5p]**

Create a function that will estimate the PageRank value for each of $N$ nodes in the graph using the stochastic adjacency matrix: $v = Mv$.

Using the damping parameter, in order to ensure normalization after each step, the formula changes as follows: $v = (1 - \alpha) \cdot v_{\text{start}} + \alpha \cdot Mv$, where $v_{\text{start}} = [\frac{1}{N}, \frac{1}{N}, \ldots, \frac{1}{N}]$.

- Start with $v = v_{\text{start}}$
- Normalize the adjacency matrix ($M$) so that each cell ($m_{ij}$) in the matrix determines the probability of going from $\text{node}_i$ to $\text{node}_j$
- At each iteration, update $v$ with a new vector $v' = (1 - \alpha) \cdot v_{\text{start}} + \alpha \cdot Mv$
- Stop the algorithm when the difference between each pair of the PageRank values for the same node in consecutive $v$ and $v'$ vectors is less than the *epsilon* parameter
- Return the result in the same way as the *networkx_pagerank* function - a list of tuples sorted in descending order of PageRank values: (node_name, node_pagerank_value)
- Verify the method for different alpha values and graphs (random graph, dead end, spider trap, complete graph)

**Hint:** in the case of *dead ends* it is necessary to modify the values in the matrix to simulate the same policy as in the *random walk* approach (*go to a random node*)

In [ ]:
def stochastic_matrix_pagerank(graph: nx.DiGraph, alpha: float = 0.85, epsilon: float = 0.0000001) -> list:
#     ...

stochastic_matrix_pagerank(G_random)

### Test the functions

**NOTE:** For some graphs, *networkx_pagerank* may have trouble generating results with $\alpha$ close to 1; in this case, note it and skip displaying this result

In [ ]:
names = ['G_random', 'G_spider_trap', 'G_dead_end', 'G_complete']
graphs = [G_random, G_spider_trap, G_dead_end, G_complete]

for name, graph in zip(names, graphs):
    print("==================")
    print("GRAPH_NAME:", name)
    
    nx.draw(graph, with_labels=True)
    plt.show()
    
    for alpha in [0.25, 0.5, 0.7, 0.85, 1.0]:
        print('ALPHA:', alpha)
        
        nx_result = networkx_pagerank(graph, alpha)
        rw_result = randomwalk_pagerank(graph, alpha)
        sm_result = stochastic_matrix_pagerank(graph, alpha)
        
        for nx_node, rw_node, sm_node in zip(nx_result, rw_result, sm_result):
            print(nx_node, rw_node, sm_node)

# Catan fandom Wikipedia

The scripts used for scraping are in the *scraper.ipynb* file.

Pages from Fandom Wikipedia: https://catan.fandom.com/wiki/Main_Page

In [ ]:
with open('catan_links.pickle', 'rb') as handle:
    d = pickle.load(handle)

G_catan = nx.DiGraph(d)

In [ ]:
nx.draw(G_catan, with_labels=True, node_size=60, font_size=8)

In [ ]:
catan_pr = networkx_pagerank(G_catan)
catan_pr

### Review example subgraph

**NOTE:** the ranking of pages in different subset of pages may differ significantly from the original, e.g. */wiki/Catan:_Traders_%26_Barbarians* which had the third highest PageRank value in the original graph

In [ ]:
number_of_pages = 20
pages_with_highest_pagerank = [page_link for page_link, pr_value in catan_pr[:number_of_pages]]

print("Pages with the highest PageRank:", pages_with_highest_pagerank)

G_sub_catan = G_catan.subgraph(pages_with_highest_pagerank)

for t in networkx_pagerank(G_sub_catan):
    print(t)
    
nx.draw(G_sub_catan, with_labels=True, node_size=60, font_size=8) 

### Task 3 - Analysis of the subset of pages about building costs [4p]

<img src="catan_building_costs.jpg" alt="drawing" width="500"/>

In [ ]:
interesting_pages = [
    '/wiki/Lumber', '/wiki/Brick', '/wiki/Wool', '/wiki/Grain', '/wiki/Ore', # Resources
    '/wiki/Road', '/wiki/Settlement', '/wiki/City', '/wiki/Development_card' # Buildings (and "build-able" Development card)
]

G_sub_buildings = G_catan.subgraph(interesting_pages).copy()

display(pd.DataFrame(nx.adjacency_matrix(G_sub_buildings).todense(), index=list(G_sub_buildings.nodes), columns=list(G_sub_buildings.nodes)))

for t in networkx_pagerank(G_sub_buildings):
    print(t)
    
nx.draw(G_sub_buildings, with_labels=True, node_size=60, font_size=8) 

**Answer the questions:**

1. **[2p]** What is the reason for *City*'s advantage over *Settlement*? Take into account how the PageRank value is calculated
2. **[2p]** Why are *Lumber* and *Brick* rated higher than other resources (*Wool*, *Ore*, *Grain*)? Take into account how the PageRank value is calculated

### Task 4 - Analysis of the subset of pages about resources and hexes [5p]

<img src="catan_hexes_resources.jpg" alt="drawing" width="500"/>

In [ ]:
interesting_pages = [
    '/wiki/Hex', '/wiki/Resource_card', # A place from which to get a resource, Resource
    '/wiki/Forest', '/wiki/Lumber', 
    '/wiki/Hill', '/wiki/Brick',
    '/wiki/Pasture', '/wiki/Wool', 
    '/wiki/Field', '/wiki/Grain', 
    '/wiki/Mountain', '/wiki/Ore',
]

G_sub_res = G_catan.subgraph(interesting_pages)

display(pd.DataFrame(nx.adjacency_matrix(G_sub_res).todense(), index=list(G_sub_res.nodes), columns=list(G_sub_res.nodes)))

for t in networkx_pagerank(G_sub_res):
    print(t)

nx.draw(G_sub_res, with_labels=True, node_size=60, font_size=8) 

**Answer the questions:**

1. **[2p]** Why are the PageRank values for the *Pasture-Wool* pair lower than for other resource's pairs (e.g. *Mountain-Ore*)? What changes to the structure of the graph will make them have comparable PageRank values?
2. **[3p]** Replace *'/wiki/Hex'* with *'/wiki/Resource_hex'*. Try to explain why the PageRank values changed a lot for each group of pages.

# Tram stops graph in Poznań

The scripts used for scraping are in the *scraper.ipynb* file.

Connections between the tram stops come from the ZTM Poznań website: https://www.ztm.poznan.pl/pl/rozklad-jazdy

In [ ]:
with open('tram_stops.pickle', 'rb') as handle:
    d = pickle.load(handle)

G_tram = nx.DiGraph(d)

In [ ]:
print(G_tram.nodes)
nx.draw(G_tram, with_labels=False, node_size=60, font_size=8)

### Task 5 - Test tram stops PageRank with different alpha values [2p]

In [ ]:
for alpha in [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99, 1.0]:
    print('ALPHA:', alpha)
    for tram_stop in stochastic_matrix_pagerank(G_tram, alpha)[:5]:
        print(tram_stop)

**Answer the question:**

1. **[2p]** Try to explain why, as the $\alpha$ value increases, the differences between the PageRank values for the *best* tram stops also increase?

### Task 6 - Subset of tram stops [6p]

In [ ]:
interesting_stops = [
    'Ogrody', 'Żeromskiego', 'Polna', 'Rynek Jeżycki', 'Kraszewskiego', 'Stare Zoo', 'Most Dworcowy',
    'Bałtyk', 'Rondo Kaponiera', 'Bukowska', 'Matejki', 'Wojskowa', 'Ostroroga', 'Rondo Nowaka-Jeziorańskiego', 
    'Arena', 'Most Teatralny', 'Poznańska', 'Fredry', 'Zamek', 'Dworzec Zachodni', 'Most Dworcowy', 'Poznań Główny',
]

G_sub_tram = G_tram.subgraph(interesting_stops)

display(pd.DataFrame(nx.adjacency_matrix(G_sub_tram).todense(), index=list(G_sub_tram.nodes), columns=list(G_sub_tram.nodes)))

for t in networkx_pagerank(G_sub_tram):
    print(t)

nx.draw(G_sub_tram, with_labels=True, node_size=60, font_size=8) 

**NOTE:** *Wojskowa* stop is an interesting case because it is available only in one direction of tram traffic

**Answer the question:**


1. **[2p]** Why does *Żeromskiego* have a comparable (even slightly higher) PageRank than *Most Dworcowy*, even though the latter has more neighbors in the graph?

2. **[4p]** Suggest a modification of the PageRank algorithm that would take into account the number of lines that pass between adjacent stops. Describe which elements of the algorithm you would modify for this purpose.

### Task 7 - TrustRank algorithm implementation [3p]

Modify the PageRank implementation from the *stochastic matrix approach*. The only change from the PageRank implementation is the assumption that in TrustRank a *random jump* to *any* page always ends in one of the *trusted* pages. For this reason, it is also necessary to provide a list of trusted nodes. Modify the mathematical formula and the algorithm implementation in such way.

In [ ]:
def trustrank(graph: nx.DiGraph, trusted_nodes: list, alpha: float = 0.85, epsilon: float = 0.0000001) -> list:
#     ...

trustrank(G_random, [0, 2])

In [ ]:
G_trustrank = nx.DiGraph()

G_trustrank.add_nodes_from(range(7))

G_trustrank.add_edge(0, 1)
G_trustrank.add_edge(2, 1)
G_trustrank.add_edge(3, 1)
G_trustrank.add_edge(4, 1)
G_trustrank.add_edge(1, 5)
G_trustrank.add_edge(5, 2)
G_trustrank.add_edge(5, 6)
G_trustrank.add_edge(6, 0)
G_trustrank.add_edge(6, 2)
G_trustrank.add_edge(0, 3)
G_trustrank.add_edge(6, 4)
    
print(G_trustrank.nodes)
print(nx.adjacency_matrix(G_trustrank).todense())
nx.draw(G_trustrank, with_labels=True)

# trustrank(G_trustrank, [3, 4])

### Task 8 - TrustRank example [4p]

For the *G_trustrank* graph above, assume that good pages are 3 and 4.

**Answer the questions:**

1. **[2p]** Why did the *trusted* nodes (3, 4) have one of the lowest TrustRank values in the graph?
2. **[2p]** What should be done to strengthen their importance?

### Task 9 - HITS algorithm implementation [5p]

Implement the **HITS** algorithm. You can choose whether you want to use the iterative version or use the eigenvector approach. Return results as two lists of tuples (similar to PageRank and TrustRank, but in two separate lists - **authorities** and **hubs**). For an iterative approach, normalize the vectors after each iteration (sum of values equals 1).

**Hint:** update the values of both vectors *simultaneously*. Use the *hubs* and *authorities* values from the previous iteration to update both vectors. Don't use an updated *hubs* vector to update *authorities* during the same iteration or vice versa.

In [ ]:
def hits(graph: nx.DiGraph, epsilon: float = 0.0000001) -> list:
#     ...

hits(G_dead_end)

### Task 10 - HITS analysis [5p]

In [ ]:
interesting_pages = [
    '/wiki/Hex', '/wiki/Resource_card', # A place from which to get a resource, Resource
    '/wiki/Forest', '/wiki/Lumber', 
    '/wiki/Hill', '/wiki/Brick',
    '/wiki/Pasture', '/wiki/Wool', 
    '/wiki/Field', '/wiki/Grain', 
    '/wiki/Mountain', '/wiki/Ore',
]

G_sub_res = G_catan.subgraph(interesting_pages)

nx.draw(G_sub_res, with_labels=True, node_size=60, font_size=8)

In [ ]:
hits(G_sub_res)

**Answer the question:**

1. **[2p]** Check the HITS values in the chart above, then change the *Hex* to *Resource_hex* once again. How did this affect the results and why?

In [ ]:
nx.draw(G_sub_tram, with_labels=True, node_size=60, font_size=8) 

In [ ]:
hits(G_sub_tram)

**Answer the question:**

2. **[3p]** Why are the results for *authorities* and *hubs* very similar?